In [1]:
from ultralytics import YOLO
import math
import numpy as np

# Load a model
model = YOLO('yolo_pose_models\yolov8l-pose.pt')

In [2]:
model_detection = YOLO("detection_models\yolo_best_single_person_247.pt")

In [3]:
def object_detection_boxes_(results_):
    print(results_[0].boxes.data.tolist())
    ball_box = []
    basket_box = []
    made_basket_box = []
    for array in results_[0].boxes.data.tolist():
        if(ball_box==[] and array[-1]==0):
            ball_box = array
        if(basket_box ==[] and array[-1]==1):
            basket_box = array
        if(made_basket_box ==[] and array[-1]==2 and array[-2]>=0.8):
            made_basket_box = array
    boxes = [ball_box, basket_box, made_basket_box]
    print(boxes)
    return boxes

In [4]:
def draw_rectangles_(frame_,boxes):
    text = ["Basket_Ball", "Hoop","Made_Basket"]
    if(boxes[0]!=[]):
        colour = (235, 206, 135)
        array = boxes[0]
        x1, y1, x2, y2 = map(int, array[:4])
        width_shrink = int((x2-x1)/6)
#         cv2.rectangle(frame_, (x1+width_shrink, y1+width_shrink), (x2-width_shrink, y2-width_shrink), (0, 255, 0), 2)
        x1, y1, x2, y2 = x1+width_shrink, y1+width_shrink, x2-width_shrink, y2-width_shrink
        cv2.rectangle(frame_, (x1, y1), (x2, y2), colour, 5)
        text_x = x1
        text_y = y1 - 10
        
        text_width, text_height = cv2.getTextSize(text[0], cv2.FONT_HERSHEY_SIMPLEX, 1.2, 2)[0]
        cv2.rectangle(frame_, (x1-3, text_y - text_height-5), (x1 + text_width+5, y1), colour, cv2.FILLED)
#         cv2.rectangle(frame_, (x1, text_y - text_height), (x1 + text_width, text_y), (255, 0, 0), cv2.FILLED)
        
        cv2.putText(frame_, text[0], (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (255, 0, 0), 5)
    if(boxes[1]!=[]):
        array = boxes[1]
        colour = (0, 255, 255)
        x1, y1, x2, y2 = map(int, array[:4])
#         width_shrink = int((x2-x1)/8)
#         cv2.rectangle(frame_, (x1+width_shrink, y1+width_shrink), (x2-width_shrink, y2-width_shrink), (0, 255, 0), 2)
        cv2.rectangle(frame_, (x1, y1), (x2, y2), colour, 5)
        text_x = x1
        text_y = y1 - 10
        
        text_width, text_height = cv2.getTextSize(text[1], cv2.FONT_HERSHEY_SIMPLEX, 1.2, 2)[0]
        cv2.rectangle(frame_, (x1-3, text_y - text_height-5), (x1 + text_width+5, y1), colour, cv2.FILLED)
        
        cv2.putText(frame_, text[1], (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (36,51,138), 5)
    if(boxes[2]!=[]):
        array = boxes[2]
        colour = 150, 75, 0
        x1, y1, x2, y2 = map(int, array[:4])
#         width_shrink = int((x2-x1)/8)
#         cv2.rectangle(frame_, (x1+width_shrink, y1+width_shrink), (x2-width_shrink, y2-width_shrink), (0, 255, 0), 2)
        cv2.rectangle(frame_, (x1, y1), (x2, y2), colour, 5)
        text_x = x1
        text_y = y1 - 10
        
        text_width, text_height = cv2.getTextSize(text[2], cv2.FONT_HERSHEY_SIMPLEX, 0.9, 2)[0]
        cv2.rectangle(frame_, (x1-3, text_y - text_height-5), (x1 + text_width+5, y1), colour, cv2.FILLED)
        
        cv2.putText(frame_, text[2], (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 4)
    return frame_
            
            

In [5]:
def output_keypoints(results_):
#     print(results_[0].keypoints.xy)
    xy_list = results_[0].keypoints.xy.tolist()
#     print(xy_list)
    return xy_list

In [6]:
def keypoints_index_on_frame(frame_, list_):
    # Iterate over the list and mark the index values on the image
    for index, point in enumerate(list_[0]):
        x, y = int(point[0]), int(point[1])
        cv2.putText(frame_, str(index), (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        cv2.circle(frame_, (x, y), 5, (0, 0, 255), -1)
    return frame_

In [7]:
def assign_keypoints_to_joints(list_):
    body_parts = {   0: 'head',
                 1: 'left_eye',
                 2: 'right_eye',
                 3: 'left_ear',
                 4: 'right_ear',
                 5: 'left_shoulder',
                 6: 'right_shoulder',
                 7: 'left_elbow',
                 8: 'right_elbow',
                 9: 'left_wrist',
                 10: 'right_wrist',
                 11: 'left_hip',
                 12: 'right_hip',
                 13: 'left_knee',
                 14: 'right_knee',
                 15: 'left_ankle',
                 16: 'right_ankle' }
    body_parts_xy = {part: [] for part in body_parts.values()}
    body_parts_xy
    for index, point in enumerate(list_[0]):
        body_parts_xy[body_parts[index]] = point
#     print(body_parts_xy)
    return body_parts_xy
    

In [8]:
def angle_of_joint(initial_joint, final_joint,body_parts_xy ):
    initial_x, initial_y = body_parts_xy[initial_joint][0], body_parts_xy[initial_joint][1]
    final_x, final_y = body_parts_xy[final_joint][0], body_parts_xy[final_joint][1]
    joint_x = -1 * (final_x - initial_x)
    joint_y = -1 * (final_y - initial_y)
    epsilon = 1e-8 
    ratio = joint_x / ((joint_x**2 + joint_y**2 + epsilon)**(1/2))
#    for tan inverse    ratio = joint_y / joint_x
    theta = math.acos(ratio)
#     for tan inverse    theta = math.atan(ratio)
    theta_deg = math.degrees(theta)
#     if(initial_joint == "left_elbow" and final_joint == "left_wrist" ):
#         print(joint_x, joint_y,  ratio, theta_deg)
#     if(initial_joint == "left_knee" and final_joint == "left_ankle" ):
#         print(joint_x, joint_y,  ratio, theta_deg)
    if(joint_y < 0):
        theta_deg = 360 - theta_deg

    return theta_deg
    
    

In [9]:
def target_joints(body_parts_xy):
    L_shoulder_to_L_elbow = angle_of_joint('left_shoulder', 'left_elbow',body_parts_xy)
    R_shoulder_to_R_elbow = angle_of_joint('right_shoulder', 'right_elbow',body_parts_xy)
    L_elbow_to_L_wrist    = angle_of_joint('left_elbow', 'left_wrist',body_parts_xy)
    R_elbow_to_R_wrist    = angle_of_joint('right_elbow', 'right_wrist',body_parts_xy)
    L_hip_to_L_knee       = angle_of_joint('left_hip', 'left_knee',body_parts_xy)
    R_hip_to_R_knee       = angle_of_joint('right_hip', 'right_knee',body_parts_xy)
    L_knee_to_L_ankle     = angle_of_joint('left_knee', 'left_ankle',body_parts_xy)
    R_knee_to_R_ankle     = angle_of_joint('right_knee', 'right_ankle',body_parts_xy)
    target_joints_dict = { 
    "L_shoulder_to_L_elbow" : L_shoulder_to_L_elbow,
    "R_shoulder_to_R_elbow" : R_shoulder_to_R_elbow,
    "L_elbow_to_L_wrist"    : L_elbow_to_L_wrist,
    "R_elbow_to_R_wrist"    : R_elbow_to_R_wrist,
    "L_hip_to_L_knee"       : L_hip_to_L_knee,
    "R_hip_to_R_knee"       : R_hip_to_R_knee ,
    "L_knee_to_L_ankle"     : L_knee_to_L_ankle,
    "R_knee_to_R_ankle"     : R_knee_to_R_ankle   }
    hands_joints_dict = { 
    "L_shoulder_to_L_elbow" : L_shoulder_to_L_elbow,
    "R_shoulder_to_R_elbow" : R_shoulder_to_R_elbow,
    "L_elbow_to_L_wrist"    : L_elbow_to_L_wrist,
    "R_elbow_to_R_wrist"    : R_elbow_to_R_wrist,}
    target_right_joints_dict = { 
    "R_shoulder_to_R_elbow" : R_shoulder_to_R_elbow,
    "R_elbow_to_R_wrist"    : R_elbow_to_R_wrist,
    "R_hip_to_R_knee"       : R_hip_to_R_knee ,
    "R_knee_to_R_ankle"     : R_knee_to_R_ankle   }
    target_left_joints_dict = { 
    "L_shoulder_to_L_elbow" : L_shoulder_to_L_elbow,
    "L_elbow_to_L_wrist"    : L_elbow_to_L_wrist,
    "L_hip_to_L_knee"       : L_hip_to_L_knee,
    "L_knee_to_L_ankle"     : L_knee_to_L_ankle}
    
    return target_joints_dict, hands_joints_dict, target_right_joints_dict, target_left_joints_dict
    

In [10]:
# def display_stats(dict_, frame_):
#     dictionary = dict_
#     position = (10, 450)  # Top right corner
#     font = cv2.FONT_HERSHEY_SIMPLEX
#     font_scale = 0.8
#     color = (255, 255, 0)  
#     thickness = 2
#     text_offset_y = position[1]
#     rect_text_offset_y = text_offset_y
#     rect_text = f"rough"
#     (rect_text_width, rect_text_height) = cv2.getTextSize(rect_text, font, font_scale, thickness)[0]
#     maxi = 0
#     for index, part in dictionary.items():
#         text = f"{index}: {part}"
#         (max_rect_text_width, max_rect_text_height) = cv2.getTextSize(text, font, font_scale, thickness)[0]
#         if(max_rect_text_width>maxi):
#             maxi = max_rect_text_width
#         rect_text_offset_y  = rect_text_offset_y  + rect_text_height + 10
#     # Iterate over the dictionary and draw the text on the frame
#     cv2.rectangle(frame_, (position[0]-10, position[1]), (maxi+20, rect_text_offset_y+10), (91,91,91), cv2.FILLED)
#     for index, part in dictionary.items():
#         text = f"{index}: {part}"
#         (text_width, text_height) = cv2.getTextSize(text, font, font_scale, thickness)[0]
#         text_offset_x = position[0] 
#         text_offset_y = text_offset_y + text_height + 10
#         cv2.putText(frame_, text, (text_offset_x, text_offset_y), font, font_scale, color, thickness)
        
#     return frame_

# def display_angle_of_ball(frame_, degrees, joints_angles):
#     font = cv2.FONT_HERSHEY_SIMPLEX
#     font_scale = 1.5
#     color = (0, 255, 0)  
#     thickness = 5
# #     position = (frame_.shape[1] - 900, 160) 
#     position = [int((joints_angles["right_wrist"][0]+joints_angles["left_wrist"][0])/2 - 600),int((joints_angles["right_wrist"][1]+joints_angles["left_wrist"][1])/2)]
#     rounded_number = round(degrees, 3)
#     text = f"release_angle: {rounded_number:.3f}"    
#     (text_width, text_height) = cv2.getTextSize(text, font, font_scale, thickness)[0]
#     cv2.putText(frame_, text, (position[0], position[1]), font, font_scale, color, thickness)
#     return frame_

In [11]:
# def display_angles_dict(frame_, dict_angles, dict_keypoints):
#     dictionary = dict_
#     position = (frame_.shape[1] - 600, 240)  # Top right corner
#     font = cv2.FONT_HERSHEY_SIMPLEX
#     font_scale = 1
#     color = (0, 0, 255)  
#     thickness = 2
#     text_offset_y = position[1]
#     # Iterate over the dictionary and draw the text on the frame
#     for index, part in dictionary.items():
#         text = f"{index}: {part}"
#         (text_width, text_height) = cv2.getTextSize(text, font, font_scale, thickness)[0]
#         text_offset_x = position[0] 
#         text_offset_y = text_offset_y + text_height + 10
#         cv2.putText(frame_, text, (text_offset_x, text_offset_y), font, font_scale, color, thickness)
#     return frame_

In [12]:
def display_angles_dict(frame_, dict_right_angles, dict_left_angles, dict_keypoints):  # Top right corner
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 1
    color = (0, 255, 0)  
    thickness = 2
    # Iterate over the dictionary and draw the text on the frame
    maxi_text_width =0
    maxi_text_height =0
    for index, part in dict_left_angles.items():
        text = f"{index}: {part:.1f} deg "
        (text_width, text_height) = cv2.getTextSize(text, font, font_scale, thickness)[0]
        if(text_width > maxi_text_width):
            maxi_text_width = text_width
        maxi_text_height = maxi_text_height + 10 + text_height
    cv2.rectangle(frame_, (int(dict_keypoints["left_hip"][0])+30,int(dict_keypoints["left_hip"][1])), (int(dict_keypoints["left_hip"][0])+30+ maxi_text_width + 20, int(dict_keypoints["left_hip"][1])+10+maxi_text_height), (0,0,0), cv2.FILLED)  
    
    position = [int(dict_keypoints["left_hip"][0])+30+10, int(dict_keypoints["left_hip"][1])]
    text_offset_x = position[0]
    text_offset_y = position[1]
    for index, part in dict_left_angles.items():
        text = f"{index}: {part:.1f} deg "
        (text_width, text_height) = cv2.getTextSize(text, font, font_scale, thickness)[0]
        text_offset_x = position[0] 
        text_offset_y = text_offset_y + text_height + 10
        cv2.putText(frame_, text, (text_offset_x,text_offset_y), font, font_scale, color, thickness)
        
        
        
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 1
    color = (0, 255, 0)  
    thickness = 2
    # Iterate over the dictionary and draw the text on the frame
    maxi_text_width =0
    maxi_text_height =0
    for index, part in dict_right_angles.items():
        text = f"{index}: {part:.1f} deg "
        (text_width, text_height) = cv2.getTextSize(text, font, font_scale, thickness)[0]
        if(text_width > maxi_text_width):
            maxi_text_width = text_width
        maxi_text_height = maxi_text_height + 10 + text_height
    cv2.rectangle(frame_, (int(dict_keypoints["right_hip"][0])-30- maxi_text_width - 20,int(dict_keypoints["right_hip"][1])), (int(dict_keypoints["right_hip"][0])-30, int(dict_keypoints["right_hip"][1])+10+maxi_text_height), (91,91,91), cv2.FILLED)  
    
    position = [int(dict_keypoints["right_hip"][0])-30- maxi_text_width - 20+10,int(dict_keypoints["right_hip"][1])]
    text_offset_x = position[0]
    text_offset_y = position[1]
    for index, part in dict_right_angles.items():
        text = f"{index}: {part:.1f} deg "
        (text_width, text_height) = cv2.getTextSize(text, font, font_scale, thickness)[0]
        text_offset_x = position[0] 
        text_offset_y = text_offset_y + text_height + 10
        cv2.putText(frame_, text, (text_offset_x,text_offset_y), font, font_scale, color, thickness)

    return frame_

In [13]:
# def flag_for_hands_acute_angles(dict_):
    

In [14]:
# # {'head': [],
# #  'left_eye': [],
# #  'right_eye': [],
# #  'left_ear': [],
# #  'right_ear': [],
# #  'left_shoulder': [],
# #  'right_shoulder': [],
# #  'left_elbow': [],
# #  'right_elbow': [],
# #  'left_wrist': [],
# #  'right_wrist': [],
# #  'left_hip': [],
# #  'right_hip': [],
# #  'left_knee': [],
# #  'right_knee': [],
# #  'left_ankle': [],
# #  'right_ankle': []}


#     target_joints_dict = { 
#     "L_shoulder_to_L_elbow" : L_shoulder_to_L_elbow,
#     "R_shoulder_to_R_elbow" : R_shoulder_to_R_elbow,
#     "L_elbow_to_L_wrist"    : L_elbow_to_L_wrist,
#     "R_elbow_to_R_wrist"    : R_elbow_to_R_wrist,
#     "L_hip_to_L_knee"       : L_hip_to_L_knee,
#     "R_hip_to_R_knee"       : R_hip_to_R_knee ,
#     "L_knee_to_L_ankle"     : L_knee_to_L_ankle,
#     "R_knee_to_R_ankle"     : R_knee_to_R_ankle   }

In [15]:
def output_boxes(results_):
    data_list = results_[0].boxes.xywh.tolist()
#     print(data_list)
    return data_list

In [16]:
def ball_center(data_list_):
    center_x = data_list_[1][0] 
    center_y = data_list_[1][1]
    return int(center_x), int(center_y)

In [17]:
def put_circle_on_ball(frame_,x,y):
    cv2.circle(frame_, (int(x), int(y)), 5, (255, 0, 0), -1)
    return frame_
    

In [18]:
def basket_center(data_list_):
    center_x = data_list_[0][0] 
    center_y = data_list_[0][1]
    return int(center_x), int(center_y)

In [19]:
def put_circle_on_basket(frame_,x,y):
    cv2.circle(frame_, (int(x), int(y)), 5, (255, 0, 0), -1)
    return frame_
    

In [20]:
def center_of_right_and_left_wrists(body_parts_xy):
    
    if(body_parts_xy["right_wrist"]!=[] and body_parts_xy["left_wrist"]!=[]):
            center_x = (body_parts_xy["right_wrist"][0] + body_parts_xy["left_wrist"][0])/2
            center_y = (body_parts_xy["right_wrist"][1] + body_parts_xy["left_wrist"][1])/2
    elif(body_parts_xy["right_wrist"]==[]):
            center_x =  body_parts_xy["left_wrist"][0]
            center_y =  body_parts_xy["left_wrist"][1]
    elif(body_parts_xy["left_wrist"]==[]):
            center_x =  body_parts_xy["right_wrist"][0]
            center_y =  body_parts_xy["right_wrist"][1]
    else:
        return False, 0, 0
    return True, int(center_x), int(center_y)
            
    

In [21]:
def put_circle_on_center_of_wrists(frame_,x,y):
    cv2.circle(frame_, (int(x), int(y)), 5, (0, 255, 0), -1)
    cv2.circle(frame_, (int(body_parts_xy["right_wrist"][0]), int(body_parts_xy["right_wrist"][1])), 5, (0, 255, 0), -1)
    cv2.circle(frame_, (int(body_parts_xy["left_wrist"][0]), int(body_parts_xy["left_wrist"][1])), 5, (0, 255, 0), -1)
    return frame_

In [22]:
def distance_between_wrists_and_ball(x_ball, y_ball, x_wrists, y_wrists):
    distance = math.sqrt((x_ball - x_wrists)**2 + (y_ball - y_wrists)**2)
    return distance
    

In [23]:
        
def display_wrists_to_ball_distance(frame_, distance):
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 1
    color = (0, 255, 0)  
    thickness = 2
    position = (frame_.shape[1] - 900, 40) 
    text = f"display_wrists_to_ball_distance: {distance}"
    (text_width, text_height) = cv2.getTextSize(text, font, font_scale, thickness)[0]
    cv2.putText(frame_, text, (position[0], position[1]), font, font_scale, color, thickness)
    return frame_
    

In [24]:
def distance_between_basket_and_ball(x_ball, y_ball, x_basket, y_basket):
    distance = math.sqrt((x_ball - x_basket)**2 + (y_ball - y_basket)**2)
    return distance

In [25]:
        
def display_basket_to_ball_distance(frame_, distance):
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 1
    color = (0, 255, 0)  
    thickness = 2
    position = (frame_.shape[1] - 900, 80) 
    text = f"display_basket_to_ball_distance: {distance}"
    (text_width, text_height) = cv2.getTextSize(text, font, font_scale, thickness)[0]
    cv2.putText(frame_, text, (position[0], position[1]), font, font_scale, color, thickness)
    return frame_

In [26]:
def intersection_over_union(box1, box2):
    # Convert xywh format to xyxy format
    x1, y1, w1, h1 = box1
    x2, y2, w2, h2 = box2
    x1_min, y1_min, x1_max, y1_max = x1 - w1/2, y1 - h1/2, x1 + w1/2, y1 + h1/2
    x2_min, y2_min, x2_max, y2_max = x2 - w2/2, y2 - h2/2, x2 + w2/2, y2 + h2/2

    # Calculate coordinates of the intersection rectangle
    inter_x_min = max(x1_min, x2_min)
    inter_y_min = max(y1_min, y2_min)
    inter_x_max = min(x1_max, x2_max)
    inter_y_max = min(y1_max, y2_max)

    # Calculate area of intersection rectangle
    inter_area = max(0, inter_x_max - inter_x_min) * max(0, inter_y_max - inter_y_min)

    # Calculate areas of individual bounding boxes
    box1_area = w1 * h1
    box2_area = w2 * h2

    # Calculate Union area
    union_area = box1_area + box2_area - inter_area

    # Calculate IoU
    if union_area > 0:
        iou = inter_area / union_area
    else:
        iou = 0

    return iou





In [27]:
def display_intersection_over_union(frame_, area, position):
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 1
    color = (255, 0, 0)  
    thickness = 2
#     position = (frame_.shape[1] - 900, 120) 
    
    text = f"display_intersection_over_union: {area}"
#     text = "MADE__BASKET"
    (text_width, text_height) = cv2.getTextSize(text, font, font_scale, thickness)[0]
    cv2.putText(frame_, text, (position[0], position[1]), font, font_scale, color, thickness)
    return frame_

In [28]:
def display_made_basket(frame_, area, position):
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 2
    color = (255, 0, 0)  
    thickness = 10
#     position = (frame_.shape[1] - 900, 120) 
    
#     text = f"display_intersection_over_union: {area}"
    text = "MADE_BASKET"
    (text_width, text_height) = cv2.getTextSize(text, font, font_scale, thickness)[0]
    cv2.putText(frame_, text, (position[0]+100, position[1]), font, font_scale, color, thickness)
    return frame_

In [29]:
def display_missed(frame_, area, position):
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 2
    color = (255, 0, 0)  
    thickness = 10
#     position = (frame_.shape[1] - 900, 120) 
    
#     text = f"display_intersection_over_union: {area}"
    text = "MISSED"
    (text_width, text_height) = cv2.getTextSize(text, font, font_scale, thickness)[0]
    cv2.putText(frame_, text, (position[0], position[1]), font, font_scale, color, thickness)
    return frame_

In [30]:
def angle_of_ball(initial_x, initial_y, final_x, final_y):
    joint_x = -1 * (final_x - initial_x)
    joint_y = -1 * (final_y - initial_y)
    epsilon = 1e-8 
    ratio = joint_x / ((joint_x**2 + joint_y**2 + epsilon)**(1/2))
    theta = math.acos(ratio)
    theta_deg = math.degrees(theta)
    if(joint_y < 0):
        theta_deg = 360 - theta_deg

    return theta_deg

In [31]:
def display_angle_of_ball(frame_, degrees, joints_angles):
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 1.5
    color = (0, 255, 0)  
    thickness = 5
#     position = (frame_.shape[1] - 900, 160) 
    position = [int((joints_angles["right_wrist"][0]+joints_angles["left_wrist"][0])/2 - 600),int((joints_angles["right_wrist"][1]+joints_angles["left_wrist"][1])/2)]
    rounded_number = round(degrees, 3)
    text = f"release_angle: {rounded_number:.3f}"    
    (text_width, text_height) = cv2.getTextSize(text, font, font_scale, thickness)[0]
    cv2.putText(frame_, text, (position[0], position[1]), font, font_scale, color, thickness)
    return frame_

In [32]:
def ball_relative_to_hoop(ball_y, basket_y):
    if ball_y >= basket_y:
        return "Below"
    else:
        return "Above"

In [33]:
def display_ball_relative_to_hoop(frame_, Flag):
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 1
    color = (0, 255, 0)  
    thickness = 2
    position = (frame_.shape[1] - 900, 200) 
    text = f"ball_relative_to_hoop: {Flag}"
    (text_width, text_height) = cv2.getTextSize(text, font, font_scale, thickness)[0]
    cv2.putText(frame_, text, (position[0], position[1]), font, font_scale, color, thickness)
    return frame_

In [34]:
def if_hands_joint_acute(hands_joints):
    all_acute = True

    for key, value in hands_joints.items():
        if value >= 80 or value <= 10 :
            all_acute = False
            break
    return all_acute

In [35]:
def annotate_list_of_balls(frame_,dict_):
    for i in range (len(dict_["ball"])):
        put_circle_on_ball(frame_, dict_["ball"][i][0] , dict_["ball"][i][1])
    return frame_
        
    


In [36]:
# # def change_from_4_points_to_2_points(array):
# #     new_array = []
# #     for sub in array:
# #         new_sub = [int((sub[0]+sub[2])/2),int((sub[1]+sub[3])/2)]
# #         new_array.append(new_sub)
# #     return new_array
# ,

In [37]:
def minimum_of_y_values(array):
    mini= array[0][1]
    for sub in array:
        if(sub[1]<mini):
            mini = sub[1]
    return mini
    

In [38]:
def projectile_equation(A,B,C,x):
#     A = 0.0024814        
#     B = -2.8373 
#     C = 869.59
    y= A*(x**2)+B*(x)+C
    tan = 2*(A)*x+B
    angle_rad = math.atan(tan)
    angle_deg = math.degrees(angle_rad)
    return  angle_deg

In [39]:
def return_statistics(data):
    
    ball_array = data["ball"]
    basket_array = data["basket"]
    person_array = data["keypoints"]

    ground_y_point = person_array[0][0]["left_ankle"][1]
    ball_initial_distance_from_gorund = ground_y_point - ball_array[0][1]
    hoop_initial_distance_from_gorund = ground_y_point - basket_array[0][1]
    person_distance_from_hoop = person_array[0][0]["left_ankle"][0] - basket_array[0][0]
    ball_distance_from_hoop = ball_array[0][0] - basket_array[0][0]
    maximum_height_achieved_by_ball = ground_y_point - minimum_of_y_values(ball_array)
    time_taken_for_projectile = len(basket_array)/30
    Range_of_projectile = ball_array[0][0] - ball_array[-1][0]
    initial_horizontal_velocity_of_ball = Range_of_projectile / time_taken_for_projectile
    final_horizontal_velocity_of_ball = initial_horizontal_velocity_of_ball
    ball_x = np.array([sublist[0] for sublist in ball_array])
#     print(ball_x)
    ball_y = np.array([sublist[1] for sublist in ball_array])
#     print(ball_y)
    z = np.polyfit(ball_x, ball_y, 2)
    A,B,C = z
    projectile_initial_angle = projectile_equation(A,B,C, ball_array[0][0])
    projectile_final_angle = projectile_equation(A,B,C, ball_array[-1][0])  
    
    initial_angle_radians = math.radians(projectile_initial_angle)
    sin_projectile_initial_angle = math.sin(initial_angle_radians)
    cos_projectile_initial_angle = math.cos(initial_angle_radians)
    
    final_angle_radians = math.radians(projectile_final_angle)
    sin_projectile_final_angle = math.sin(final_angle_radians)
    cos_projectile_final_angle = math.cos(final_angle_radians)
   
    initial_velocity_of_ball = initial_horizontal_velocity_of_ball/ cos_projectile_initial_angle
    final_velocity_of_ball = final_horizontal_velocity_of_ball/ cos_projectile_final_angle 

    initial_vertical_velocity_of_ball = initial_velocity_of_ball * sin_projectile_initial_angle
    final_vertical_velocity_of_ball = final_velocity_of_ball * sin_projectile_final_angle
    

    stats = {"ball_initial_distance_from_gorund  ": ball_initial_distance_from_gorund,
          "hoop_initial_distance_from_gorund  ": hoop_initial_distance_from_gorund,
          "person_distance_from_hoop  ": person_distance_from_hoop,
          "ball_distance_from_hoop  ": ball_distance_from_hoop,
          "time_taken_for_projectile  ": time_taken_for_projectile,
          "Range_of_projectile  ": Range_of_projectile,
          "maximum_height_achieved_by_ball  ":maximum_height_achieved_by_ball,
          "projectile_initial_angle  ": projectile_initial_angle, 
          "initial_horizontal_velocity_of_ball  ": initial_horizontal_velocity_of_ball,
          "initial_vertical_velocity_of_ball  ": initial_vertical_velocity_of_ball ,
          "initial_velocity_of_ball  ":initial_velocity_of_ball,
          "projectile_final_angle  ":projectile_final_angle,
          "final_horizontal_velocity_of_ball  ":final_horizontal_velocity_of_ball,
          "final_vertical_velocity_of_ball  ":final_vertical_velocity_of_ball,
          "final_velocity_of_ball  ":final_velocity_of_ball,}
    stats_required = {
          "Shot arc angle at release": projectile_initial_angle,
          "Shot arc angle at entry": -1*(projectile_final_angle),
          "Shot arc peak": maximum_height_achieved_by_ball * 0.004075425790754258,
          "Time from release to rim": time_taken_for_projectile,
          "Speed at release": initial_velocity_of_ball * 0.004075425790754258}
    
    stats = {key: round(value, 1) for key, value in stats_required.items()}


#     statistics = [ball_initial_distance_from_gorund, hoop_initial_distance_from_gorund, person_distance_from_hoop, ball_distance_from_hoop, maximum_height_achieved_by_ball,time_taken_for_projectile, Range_of_projectile, initial_horizontal_velocity_of_ball, projectile_initial_angle, projectile_final_angle, initial_velocity_of_ball, final_velocity_of_ball]
#     return ball_array, basket_array, person_array, statistics
    return stats

In [40]:
def display_stats(dict_, frame_):
    dictionary = dict_
    position = (10, 450)  # Top right corner
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 1
    color = (255, 255, 0)  
    thickness = 2
    text_offset_y = position[1]
    rect_text_offset_y = text_offset_y
    rect_text = f"rough"
    (rect_text_width, rect_text_height) = cv2.getTextSize(rect_text, font, font_scale, thickness)[0]
    maxi = 0
    ext = [" deg"," deg"," m"," sec"," m/s"]
    c=0
    for index, part in dictionary.items():
        text = f"{index}: {part}{ext[c]}"
        c=c+1
        (max_rect_text_width, max_rect_text_height) = cv2.getTextSize(text, font, font_scale, thickness)[0]
        if(max_rect_text_width>maxi):
            maxi = max_rect_text_width
        rect_text_offset_y  = rect_text_offset_y  + rect_text_height + 10
    # Iterate over the dictionary and draw the text on the frame
    cv2.rectangle(frame_, (position[0]-10, position[1]), (maxi+20, rect_text_offset_y+10), (91,91,91), cv2.FILLED)
    ext = [" deg"," deg"," m"," sec"," m/s"]
    c=0
    for index, part in dictionary.items():
        text = f"{index}: {part}{ext[c]}"
        c=c+1
        (text_width, text_height) = cv2.getTextSize(text, font, font_scale, thickness)[0]
        text_offset_x = position[0] 
        text_offset_y = text_offset_y + text_height + 10
        cv2.putText(frame_, text, (text_offset_x, text_offset_y), font, font_scale, color, thickness)
        
    return frame_

In [50]:
import cv2
from ultralytics import YOLO



# Open the video file
video_path = "input_videos\input_video.mp4"
cap = cv2.VideoCapture(video_path)

# Get video properties
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
output_path = "input_videos\currently_input_video.mp4"

# Create video writer object
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
count_frames_ = 0
free_throw_count = 0
success_count = 0
failure_count = 0

made_basket_threshold_intersection_over_union = 0.2

flag_hands_joints_acute = False
flag_ball_angle_acute = False
# flag_ball_wrist_distance = False
flag_ball_release = False
# flag_made_basket = False
flag_free_throw_start = False
# flag_made_basket = False
flag_free_throw_end = True
made_basket_list = []
basket_review_frames_count = 0
basket_review_flag = False
data_collect_start_point = False
data_collect_end_point = True
data_global_array = []
count_pose_display_ = 0
display_pose_ = False
basket_review_frames_count_t = 90
count_pose_display_t =90
# Loop through the video frames
while cap.isOpened():
    # Read a frame from the video
    success, frame = cap.read()

    if success:
        # Run YOLOv8 object detection inference on the frame, input is frame and output is results of yolo model
        results_detection = model_detection.track(frame) 
        
        objects_boxes_ = object_detection_boxes_(results_detection)
        
        
        data = output_boxes(results_detection)
#         annotated_frame = results_detection[0].plot()
        

        
        
        results = model.track(frame) 
        results_list = output_keypoints(results)
        # Visualize the results on the frame
        annotated_frame = results[0].plot()
        annotated_frame = keypoints_index_on_frame(annotated_frame, results_list)
        annotated_frame = draw_rectangles_(annotated_frame, objects_boxes_)
        
        if(len(data)<2):
            print("skipped the frame : ********************************************************")
            text_upper = f"total_attempts_made : {free_throw_count}     Success : {success_count}     Failure : {failure_count}"
            colour = (235, 206, 135)
            font = cv2.FONT_HERSHEY_SIMPLEX
            font_scale = 1.2
            thickness =5

            text_width, text_height = cv2.getTextSize(text_upper, cv2.FONT_HERSHEY_SIMPLEX, 1.2, 5)[0]
            cv2.rectangle(annotated_frame, (int((1780-text_width)/2), 10), (int((1780-text_width)/2) + text_width + 20, 10 + text_height + 10 + 10), (91,91,91), cv2.FILLED)
            cv2.putText(annotated_frame, text_upper, (int((1780-text_width)/2)+10, 10 + text_height + 10), font, font_scale, (255,0,255), thickness)
            
            if(basket_review_frames_count<90 and basket_review_flag):
                basket_review_frames_count = basket_review_frames_count + 1
                if(made_basket_list[-1]=="False"):
                    annotated_frame = display_missed(annotated_frame, area_intersection_over_union, [basket_center_x, basket_center_y])
                else:
                    annotated_frame = display_made_basket(annotated_frame, area_intersection_over_union, [basket_center_x, basket_center_y])
                if(data_collect_start_point == False and data_collect_end_point == True):
                    stats = return_statistics( data_global_array[-1])
    #                 print(data_global_array[-1])
                    annotated_frame = display_stats(stats, annotated_frame)
            else:
                basket_review_flag = False
                basket_review_frames_count = 0 
                
            
            out.write(annotated_frame)
            count_frames_ = count_frames_ + 1
            print("count_frames_", count_frames_)
            continue
        
      
        
   
        points_of_joints = assign_keypoints_to_joints(results_list)
        print(points_of_joints)
        angles_of_joints, hands_joints, right_joints, left_joints = target_joints(points_of_joints)
#         print(angles_of_joints)
        
        flag_hands_joints_acute = if_hands_joint_acute(hands_joints)
    
#         annotated_frame = display_angles_dict(annotated_frame, angles_of_joints)        
        
        

        
        ball_center_x, ball_center_y = ball_center(data)
        
        basket_center_x, basket_center_y = basket_center(data)
        
#         annotated_frame = put_circle_on_ball(annotated_frame, ball_center_x , ball_center_y)
        
#         annotated_frame = put_circle_on_basket(annotated_frame, basket_center_x , basket_center_y)
        
        flag_left_right_wrist, center_x_wrists, center_y_wrists = center_of_right_and_left_wrists(points_of_joints)
#         print(flag_left_right_wrist, center_x_wrists, center_y_wrists, body_parts_xy["right_wrist"], body_parts_xy["left_wrist"])
        
#         annotated_frame = put_circle_on_center_of_wrists(annotated_frame, center_x_wrists , center_y_wrists)
        
        distance_ball_to_wrists = distance_between_wrists_and_ball(ball_center_x, ball_center_y, center_x_wrists, center_y_wrists)
#         print(distance_ball_to_wrists)
#         if(distance_ball_to_wrists> 50):
#             flag_ball_wrist_distance = True
            
                                  
#         annotated_frame = display_wrists_to_ball_distance(annotated_frame, distance_ball_to_wrists)
  
        distance_ball_to_basket = distance_between_basket_and_ball(ball_center_x, ball_center_y, basket_center_x, basket_center_y)
        
#         annotated_frame = display_basket_to_ball_distance(annotated_frame, distance_ball_to_basket)
 
        area_intersection_over_union = intersection_over_union(data[0], data[1])
    
#         annotated_frame =  display_intersection_over_union(annotated_frame, area_intersection_over_union)
        
        flag_ball_relative_to_hoop = ball_relative_to_hoop(ball_center_y, basket_center_y)
        
#         annotated_frame =  display_ball_relative_to_hoop(annotated_frame , flag_ball_relative_to_hoop)
        
        
        if(count_frames_ > 0):
            angle_ = angle_of_ball(prev_ball_center_x, prev_ball_center_y, ball_center_x, ball_center_y )
#             annotated_frame = display_angle_of_ball(annotated_frame , angle_)
            if(angle_>10 and angle_<90):
                flag_ball_angle_acute = True
            else: 
                flag_ball_angle_acute = False
            
            if(distance_ball_to_basket > 200 and flag_ball_relative_to_hoop == "Below" and angle_ > 180):
                flag_free_throw_end = True
                flag_free_throw_start = False
                if(len(made_basket_list)<free_throw_count):
                    failure_count = failure_count +1
                    made_basket_list.append("False")
                    basket_review_flag = True
                
            
        
        
#         if(flag_hands_joints_acute and flag_ball_angle_acute and flag_ball_wrist_distance and not flag_free_throw_start):
            
        if(flag_hands_joints_acute and flag_ball_angle_acute and distance_ball_to_wrists>50 and not flag_free_throw_start):
            flag_free_throw_start = True
            display_pose_ = True
            flag_free_throw_end = False
            after_Release_angle = angle_
            print(after_Release_angle)
            
            after_release_keypoints = points_of_joints
            print("print(after_release_keypoints) :",after_release_keypoints)
            after_release_pose  = angles_of_joints 
            after_release_right_pose = right_joints
            after_release_left_pose = left_joints
            free_throw_count = free_throw_count + 1
            data_collect_start_point = True 
            data_collect_end_point = False
            data_local_array = {"ball" : [], "keypoints" : [], "basket" : []}
            
        if(data_collect_start_point ==True and data_collect_end_point == False and flag_free_throw_start and not flag_free_throw_end and flag_ball_relative_to_hoop == "Above" and count_frames_ > 0 and (angle_>90 and angle_<270)):
            data_collect_start_point = False
            data_collect_end_point = True
            data_global_array.append(data_local_array)
            
#         if(data_collect_start_point == True and data_collect_end_point == False):
#             if(len(data_local_array)==0):
#                 data_local_array.append([[prev_ball_center_x, prev_ball_center_y],[ball_center_x, ball_center_y ],[basket_center_x, basket_center_y],[angles_of_joints]])
#                 annotated_frame = put_circle_on_ball(annotated_frame, prev_ball_center_x, prev_ball_center_y)
#                 annotated_frame = put_circle_on_ball(annotated_frame, ball_center_x , ball_center_y)
#             else:
#                 data_local_array.append([[ball_center_x, ball_center_y ],[basket_center_x, basket_center_y],[angles_of_joints]])
#                 annotated_frame = put_circle_on_ball(annotated_frame, ball_center_x , ball_center_y)


        if(data_collect_start_point == True and data_collect_end_point == False and flag_free_throw_start and not flag_free_throw_end):
            if(len(data_local_array["ball"])==0):
                data_local_array["ball"].append([prev_ball_center_x, prev_ball_center_y])
                data_local_array["ball"].append([ball_center_x, ball_center_y])
                data_local_array["basket"].append([basket_center_x, basket_center_y])
                data_local_array["keypoints"].append([after_release_keypoints])
                annotated_frame = annotate_list_of_balls(annotated_frame, data_local_array)

            else:
                data_local_array["ball"].append([ball_center_x, ball_center_y])
                data_local_array["basket"].append([basket_center_x, basket_center_y])
                print("ll,     ",after_release_keypoints)
                data_local_array["keypoints"].append([after_release_keypoints]) 
                annotated_frame = annotate_list_of_balls(annotated_frame, data_local_array)

            

        prev_ball_center_x, prev_ball_center_y = ball_center_x, ball_center_y
#             flag_ball_release = True
#         print(flag_hands_joints_acute, flag_ball_angle_acute, flag_ball_wrist_distance)
        if(free_throw_count == 4):
            count_pose_display_t =1000
        else:
            count_pose_display_t =90
        if(display_pose_ and count_frames_ > 0 and count_pose_display_<count_pose_display_t):
            annotated_frame = display_angles_dict(annotated_frame, after_release_right_pose, after_release_left_pose, points_of_joints) 
            count_pose_display_ = count_pose_display_+1
            if(count_pose_display_ ==count_pose_display_t):
                count_pose_display_ = 0
                display_pose_ = False
        if(flag_free_throw_start and not flag_free_throw_end and count_frames_ > 0):  
#             annotated_frame = display_angle_of_ball(annotated_frame ,  after_Release_angle, points_of_joints)
            
            if(area_intersection_over_union > made_basket_threshold_intersection_over_union and distance_ball_to_basket < 50):
                if(len(made_basket_list)<free_throw_count):
                    made_basket_list.append("True")
                    success_count = success_count+1
                    basket_review_flag = True
#                 flag_made_basket = True
#                 annotated_frame = display_made_basket(annotated_frame, area_intersection_over_union, [basket_center_x, basket_center_y])
                
#         if(free_throw_count>0 and flag_free_throw_start == False and flag_free_throw_end == True and made_basket_list[free_throw_count-1] == "False"):
#             annotated_frame = display_missed(annotated_frame, area_intersection_over_union, [basket_center_x, basket_center_y])
        if(free_throw_count == 4):
            basket_review_frames_count_t = 1000
        else:
            basket_review_frames_count_t = 90
        if(basket_review_frames_count<basket_review_frames_count_t and basket_review_flag):
            basket_review_frames_count = basket_review_frames_count + 1
            if(made_basket_list[-1]=="False"):
                annotated_frame = display_missed(annotated_frame, area_intersection_over_union, [basket_center_x, basket_center_y])
            else:
                annotated_frame = display_made_basket(annotated_frame, area_intersection_over_union, [basket_center_x, basket_center_y])
            if(data_collect_start_point == False and data_collect_end_point == True):
                stats = return_statistics( data_global_array[-1])
#                 print(data_global_array[-1])
                annotated_frame = display_stats(stats, annotated_frame)
        else:
            basket_review_flag = False
            basket_review_frames_count = 0 
            
            
        text_upper = f"total_attempts_made : {free_throw_count}     Success : {success_count}     Failure : {failure_count}"
        colour = (235, 206, 135)
        font = cv2.FONT_HERSHEY_SIMPLEX
        font_scale = 1.2
        thickness =5
        
        text_width, text_height = cv2.getTextSize(text_upper, cv2.FONT_HERSHEY_SIMPLEX, 1.2, 5)[0]
        cv2.rectangle(annotated_frame, (int((1780-text_width)/2), 10), (int((1780-text_width)/2) + text_width + 20, 10 + text_height + 10 + 10), (91,91,91), cv2.FILLED)
        cv2.putText(annotated_frame, text_upper, (int((1780-text_width)/2)+10, 10 + text_height + 10), font, font_scale, (255,0,255), thickness)
        
            
            
            
            
            
        # Save the annotated frame to the output video
        annotated_frame = cv2.resize(annotated_frame, (1280, 720))
        out.write(annotated_frame)
        annotated_frame = cv2.resize(annotated_frame, (900, 540)) 

        # Display the annotated frame
        cv2.imshow("inference : ",annotated_frame)
        count_frames_ = count_frames_ + 1
        print(free_throw_count)
        print("count_frames_", count_frames_)
#         if(count_frames_>5):
#             break

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    else:
        # Break the loop if the end of the video is reached
        break

# Release the video capture object, video writer, and close the display window
cap.release()
out.release()
cv2.destroyAllWindows()



0: 384x640 1 basket_ball, 1 hoop, 1312.5ms
Speed: 4.0ms preprocess, 1312.5ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)



[[331.94793701171875, 184.362060546875, 487.7422180175781, 355.18243408203125, 1.0, 0.9076841473579407, 1.0], [1019.230712890625, 639.455078125, 1148.77734375, 761.4788208007812, 2.0, 0.839458167552948, 0.0]]
[[1019.230712890625, 639.455078125, 1148.77734375, 761.4788208007812, 2.0, 0.839458167552948, 0.0], [331.94793701171875, 184.362060546875, 487.7422180175781, 355.18243408203125, 1.0, 0.9076841473579407, 1.0], []]


0: 384x640 1 person, 932.8ms
Speed: 4.0ms preprocess, 932.8ms inference, 4.0ms postprocess per image at shape (1, 3, 640, 640)



{'head': [1106.6475830078125, 582.4432373046875], 'left_eye': [1111.0621337890625, 572.4462890625], 'right_eye': [1104.7645263671875, 574.1786499023438], 'left_ear': [1134.703369140625, 566.7716674804688], 'right_ear': [1115.0701904296875, 571.4441528320312], 'left_shoulder': [1169.0042724609375, 611.2044067382812], 'right_shoulder': [1126.117431640625, 611.125732421875], 'left_elbow': [1174.3553466796875, 684.9822387695312], 'right_elbow': [1129.1939697265625, 671.491455078125], 'left_wrist': [1115.140625, 709.3873901367188], 'right_wrist': [1104.2596435546875, 681.37939453125], 'left_hip': [1171.2999267578125, 730.206298828125], 'right_hip': [1139.3052978515625, 727.0321044921875], 'left_knee': [1170.76220703125, 835.4365844726562], 'right_knee': [1128.962646484375, 828.249755859375], 'left_ankle': [1184.291748046875, 942.0982055664062], 'right_ankle': [1140.7235107421875, 915.4443359375]}
0
count_frames_ 1


0: 384x640 1 basket_ball, 1 hoop, 1400.7ms
Speed: 4.0ms preprocess, 1400.7ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)



...

{'head': [1104.937744140625, 583.3511962890625], 'left_eye': [1109.53857421875, 573.8068237304688], 'right_eye': [1104.1756591796875, 575.5652465820312], 'left_ear': [1133.9947509765625, 567.5759887695312], 'right_ear': [1115.74853515625, 572.1778564453125], 'left_shoulder': [1167.7974853515625, 610.8004150390625], 'right_shoulder': [1122.7623291015625, 611.3699340820312], 'left_elbow': [1173.4422607421875, 685.63818359375], 'right_elbow': [1125.46484375, 674.350830078125], 'left_wrist': [1113.0211181640625, 715.79296875], 'right_wrist': [1102.7833251953125, 701.85498046875], 'left_hip': [1173.564208984375, 731.1456298828125], 'right_hip': [1140.09326171875, 729.3116455078125], 'left_knee': [1168.9703369140625, 834.063232421875], 'right_knee': [1128.397216796875, 831.996337890625], 'left_ankle': [1184.6954345703125, 942.6798706054688], 'right_ankle': [1139.2362060546875, 919.041015625]}
0
count_frames_ 3


0: 384x640 1 basket_ball, 1 hoop, 1372.8ms
Speed: 4.0ms preprocess, 1372.8ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)



[[332.87542724609375, 185.20706176757812, 487.0720520019531, 356.1080322265625, 1.0, 0.9084905385971069, 1.0], [1017.353515625, 683.802734375, 1146.125, 800.8093872070312, 2.0, 0.8226699233055115, 0.0]]
[[1017.353515625, 683.802734375, 1146.125, 800.8093872070312, 2.0, 0.8226699233055115, 0.0], [332.87542724609375, 185.20706176757812, 487.0720520019531, 356.1080322265625, 1.0, 0.9084905385971069, 1.0], []]


0: 384x640 1 person, 924.6ms
Speed: 4.0ms preprocess, 924.6ms inference, 4.0ms postprocess per image at shape (1, 3, 640, 640)


{'head': [1104.9669189453125, 583.5344848632812], 'left_eye': [1109.4970703125, 574.0167236328125], 'right_eye': [1103.8714599609375, 575.9571533203125], 'left_ear': [1134.0174560546875, 567.6061401367188], 'right_ear': [1114.5284423828125, 572.7886962890625], 'left_shoulder': [1168.40966796875, 610.6447143554688], 'right_shoulder': [1121.8052978515625, 611.4312133789062], 'left_elbow': [1173.82763671875, 685.4905395507812], 'right_elbow': [1125.2027587890625, 674.0579833984375], 'left_wrist': [1116.150634765625, 716.6698608398438], 'right_wrist': [1103.193115234375, 705.3986206054688], 'left_hip': [1175.142822265625, 730.1990966796875], 'right_hip': [1140.5179443359375, 728.4609985351562], 'left_knee': [1169.31103515625, 833.553955078125], 'right_knee': [1127.58740234375, 831.4179077148438], 'left_ankle': [1184.640380859375, 942.5903930664062], 'right_ankle': [1138.4876708984375, 919.2510375976562]}
0
count_frames_ 4


In [ ]:
made_basket_list

In [ ]:
len(data_global_array[0]["pose"])

In [51]:
# Release the video capture object, video writer, and close the display window
cap.release()
out.release()
cv2.destroyAllWindows()

In [ ]:
data_global_array[0]

In [ ]:
points_of_joints

In [ ]:
len(data_global_array[3]["ball"])

In [ ]:
len(global_array_data_saved)

In [ ]:
# import cv2
# import torch

# # Define the tensor
# tensor = torch.tensor([[[1106.3483, 583.2304],
#                         [1110.6146, 573.4428],
#                         [1104.5686, 574.7374],
#                         [1134.0538, 566.9627],
#                         [1115.0956, 570.5521],
#                         [1168.0262, 611.2357],
#                         [1126.2949, 608.9180],
#                         [1172.8837, 685.9680],
#                         [1132.8390, 668.4886],
#                         [1112.6084, 711.7311],
#                         [1108.4567, 683.7622],
#                         [1171.2192, 730.7853],
#                         [1139.7151, 726.9801],
#                         [1170.5370, 835.0827],
#                         [1128.8016, 828.9874],
#                         [1183.8893, 942.6035],
#                         [1141.4540, 916.9026]]])

# # Convert the tensor to a list
# tensor_list = tensor.tolist()

# # Load an image
# image_path = "path/to/image.jpg"  # Replace with the path to your image
# image = cv2.imread(image_path)

# # Iterate over the list and mark the index values on the image
# for index, point in enumerate(tensor_list[0]):
#     x, y = int(point[0]), int(point[1])
#     cv2.putText(image, str(index), (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
#     cv2.circle(image, (x, y), 5, (0, 0, 255), -1)

# # Display the image
# cv2.imshow("Image with Index Values", image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()


In [ ]:
1165-343

In [ ]:
3.35/822